
# Aplicación de herramientas de clase



In [12]:
library(readr)
library(dplyr)
data<-read.csv("data.csv") #retomamos la base de datos creada en el script de limpieza


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



## Pruebas de hipótesis interesantes

In [ ]:
#privada vs imss vs isste


#indigena afecta

# Hipertesion

## diabetes obesidad
## tabaquismo


¿Existe un efecto diferenciado de la mortalidad frente al imss, isste e institución privada?


In [63]:
source<-data %>% 
select(resultado_lab,fecha_def,sector) %>% 
filter (resultado_lab=="POSITIVO A SARS-COV-2")#

#aqui cuento la cantidad de personas atendidas en cada sector
people_by_hospital<-source%>% 
group_by(sector) %>% 
summarize(count_pplo=n())
#filter(fecha_def != "NA" )

#Aqui asumo que la persona murio si solo si hay fecha de defuncion, con base en eso calculo la cantidad de fallecidos
death_by_hospital<-source %>% 
filter(fecha_def != "NA" )%>% 
group_by(sector) %>% 
summarize(count_dths=n())

#match
(letalidad<-merge(death_by_hospital,people_by_hospital, by="sector") %>% 
 mutate(letalidad=count_dths/count_pplo))

##letalidad global
(global_letality<-sum(letalidad$count_dths)/sum(letalidad$count_pplo))


sector,count_dths,count_pplo,letalidad
CRUZ ROJA,1,87,0.01149425
ESTATAL,41,246,0.16666667
IMSS,19553,84291,0.23197020
ISSSTE,2335,7660,0.30483029
PEMEX,994,5574,0.17832795
PRIVADA,446,8520,0.05234742
SEDENA,536,2842,0.18859958
SEMAR,230,3042,0.07560815
SSA,8218,243092,0.03380613


[1] 0.09104724

[1] 2945.742

[1] 51.74495

[1] 0.08275764

Prueba de hipotesis:

¿Es letalidad en el sector j mayor a la letalidad del país?

$H_n: \mu_j=.091$

$H_a: \mu_j >.091$

In [65]:
#Asumimos que la probabilidad 

letalidad<-letalidad %>% 
mutate (esperanza=count_pplo*letalidad ,st_let = sqrt(count_pplo*(1-letalidad)))%>% 
mutate(z = (letalidad*count_pplo-global_letality*count_pplo)/sqrt(global_letality*count_pplo*(1-global_letality)))


Para la prueba de hipótesis asumiremos que la distribución de deceso es binomial y partiendo de eso aprovecharemos el teorema del limite central poder realizar las pruebas de hipotesis con base en el estadístico Z.

$Z=\frac{p-n\pi}{\sqrt{n\pi(1-\pi)}}$

donde $p=$ letalidad $\cdot n$

In [84]:
limit<-qnorm(.995,0,1) # for alpha=.01 one talied
letalidad<-
letalidad %>% mutate(prueba=ifelse(z>limit,"Se rechaza H0", "No se rechaza H0"))

In [85]:
letalidad %>% select (sector, z, prueba)

sector,z,prueba
CRUZ ROJA,-2.579361,No se rechaza H0
ESTATAL,4.122844,Se rechaza H0
IMSS,142.222504,Se rechaza H0
ISSSTE,65.040461,Se rechaza H0
PEMEX,22.651540,Se rechaza H0
PRIVADA,-12.417229,No se rechaza H0
SEDENA,18.077805,Se rechaza H0
SEMAR,-2.960037,No se rechaza H0
SSA,-98.104586,No se rechaza H0


# Notas
resultado_lab=="POSITIVO A SARS-COV-2"
fecha_def is not NA
sector